In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file using 'from dotenv import find_dotenv, load_dotenv'
load_dotenv(find_dotenv(filename='SURF-Project_Optimizing-PerunaBot/setup/.env'))

openai_api_key = os.environ['OPENAI_API_KEY']

True

In [2]:
#langsmith
from langsmith import Client
import os

langsmith_api_key = os.environ["LANGSMITH_API_KEY"]
langchain_endpoint = os.environ["LANGCHAIN_ENDPOINT"]
langsmith_project = os.environ["LANGCHAIN_PROJECT"]
os.environ["LANGCHAIN_TRACING_V2"]

# Initialize LangSmith Client using 'from langsmith import Client'
langsmith_client = Client()

In [17]:
from OG_PerunaBot_chain import Original_PerunaBot_eval_chain
from chain_0 import base_retriever_eval_chain_0
from chain_1 import parent_retriever_eval_chain_1
from chain_2 import ensemble_retriever_eval_chain_2

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
from ragas.integrations.langchain import EvaluatorChain
from langchain.smith import run_on_dataset, RunEvalConfig
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

# create evaluation chains
faithfulness_chain = EvaluatorChain(metric=faithfulness)
answer_rel_chain = EvaluatorChain(metric=answer_relevancy)
context_rel_chain = EvaluatorChain(metric=context_precision)
context_recall_chain = EvaluatorChain(metric=context_recall)


# Wrap the RAGAS metrics to use in LangChain
evaluators = [
    faithfulness_chain,
    answer_rel_chain,
    context_rel_chain,
    context_recall_chain,
]

eval_config = RunEvalConfig(custom_evaluators=evaluators)

# datasets in langsmith
data_set_1 = "SMU Schools Basic Info"
data_set_2 = "SMU Campus Facts"
project_name = "First test eval for "


In [ ]:

# Original PerunaBot chain on dataset 1
chain_results = run_on_dataset(
    client=langsmith_client,
    llm_or_chain_factory=Original_PerunaBot_eval_chain,
    dataset_name=data_set_1,
    verbose=True,
    evaluation=eval_config,
    project_name=project_name + "Original PerunaBot chain",
    project_metadata={
            "chain": "Original_PerunaBot_eval_chain",
            "dataset": data_set_1,
            "version": "0.1"
        }
)

In [ ]:
import shelve
# Load the LangChain documents from the shelve file
with shelve.open("../Common/serialized_data/data_preprocessing_langchain_docs.db") as db:
    langchain_docs_loaded = {key: db[key] for key in db}

normal_split_docs = langchain_docs_loaded["normal_split_docs"]
print(len(normal_split_docs))

In [18]:
from openai import embeddings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm= ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
     generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings, 
)

# Generate testset
testset = generator.generate_with_langchain_docs(normal_split_docs, test_size=25, distributions = {simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/14646 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-Hkbsf8yAWp77cZYrVZcR2Dim on tokens per min (TPM): Limit 60000, Used 59901, Requested 375. Please try again in 276ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}